plot some example ROIs that are stable place cells throughout first 6 days of imaging
plot cell footprint on day1 and day6 with cell highlighted and red green overlay as red and purple, union is white

In [1]:
import os
import math

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import gridspec
import dill

import suite2p as s2p

import TwoPUtils as tpu
import STX3KO_analyses as stx
from STX3KO_analyses import utilities as u

ko_mice = stx.ymaze_sess_deets.ko_mice
ctrl_mice = stx.ymaze_sess_deets.ctrl_mice


%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
figdir = "/mnt/BigDisk/YMazeResults/neural/example_place_cells"
os.makedirs(figdir, exist_ok=True)

In [3]:
print(ctrl_mice)

('4467331.1', '4467331.2', '4467332.1', '4467332.2', '4467333.1', 'mCherry6', 'mCherry7', 'mCherry8', 'mCherry9')


In [4]:
sess = u.single_mouse_concat_sessions(ctrl_mice[6], date_inds = np.arange(6),load_stats=True, load_ops = True)

/home/mplitt/mambaforge/envs/STX3/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


14_10_2021 YMaze_LNovel
15_10_2021 YMaze_LNovel
16_10_2021 YMaze_LNovel
17_10_2021 YMaze_LNovel
18_10_2021 YMaze_LNovel
19_10_2021 YMaze_LNovel


In [5]:
len(sess.s2p_stats)

6

In [6]:

# pc_mask = sess.fam_place_cell_mask() + sess.nov_place_cell_mask() >0
pc_mask =  sess.nov_place_cell_mask() >0
pc_inds = np.argwhere(pc_mask)[:,0]

In [7]:
sess.novel_arm


-1

In [ ]:
save =False
plt.ioff()
for cell in pc_inds:
    fig = plt.figure(figsize = [20, 5])
    gs = gridspec.GridSpec(6, 5, figure=fig)
    mean = np.nanmean(sess.trial_matrices['spks'][:,:,cell].ravel())
    for day in range(6):
        fam_mask = (sess.trial_info['sess_num']==day) & (sess.trial_info['LR']==-1*sess.novel_arm)
        nov_mask = (sess.trial_info['sess_num']==day) & (sess.trial_info['LR']==sess.novel_arm)
        fam_trialmat = sess.trial_matrices['spks'][fam_mask,:,:][:,:,cell]
        nov_trialmat = sess.trial_matrices['spks'][nov_mask,:,:][:,:,cell]
        
    
        fam_trialmat[np.isnan(fam_trialmat)]=1E-3
        nov_trialmat[np.isnan(nov_trialmat)]=1E-3
        
        day_inds = sess.trial_info['sess_num'][sess.trial_info['LR']==-1*sess.novel_arm]==day
        fam_ax = fig.add_subplot(gs[day,0])
        h = fam_ax.imshow(fam_trialmat/mean, cmap = 'magma', aspect='auto', vmin = 0, vmax=5)
        
        nov_ax = fig.add_subplot(gs[day,1])
        nov_ax.imshow(nov_trialmat/mean, cmap = 'magma', aspect='auto', vmin = 0, vmax=5)
        
        if day<5:
            fam_ax.set_xticks([])
            nov_ax.set_xticks([])
        else:
            fam_ax.set_xticks((-.5, 9.5, 19.5, 29.5))
            fam_ax.set_xticklabels((0,100,200,300))
            
            nov_ax.set_xticks((-.5, 9.5, 19.5, 29.5))
            nov_ax.set_xticklabels((0,100,200,300))
    
    
    cbar_ax = fig.add_subplot(gs[2:4,2])
    plt.colorbar(h,ax=cbar_ax)
    
    day0_com = (sess.s2p_stats[0][cell]['xpix'].mean(), sess.s2p_stats[0][cell]['ypix'].mean())
    day5_com = (sess.s2p_stats[len(sess.s2p_stats)-1][cell]['xpix'].mean(), sess.s2p_stats[len(sess.s2p_stats)-1][cell]['ypix'].mean())
    
    day0img = np.zeros([100,100,3])

    x_edges = (int(day0_com[1]-50), int(day0_com[1]+50))
    y_edges = ( int(day0_com[0]-50), int(day0_com[0]+50))
    g_img = sess.s2p_ops[0]['meanImg'][x_edges[0]:x_edges[1], y_edges[0]:y_edges[1]]
    r_img = sess.s2p_ops[0]['meanImg_chan2'][x_edges[0]:x_edges[1], y_edges[0]:y_edges[1]]
    day0img[:,:,1] = g_img/np.amax(g_img)
    day0img[:,:,0] = r_img/np.amax(r_img)
    day0img[:,:,2] = r_img/np.amax(r_img)
    
    day0_ax = fig.add_subplot(gs[1:5,3])
    day0_ax.imshow(day0img,cmap='Greys_r')
    
    circle = plt.Circle((50,50), 7, fill=False, color='blue',linewidth=3)
    day0_ax.add_patch(circle)
    
    day5img = np.zeros([100,100,3])

    x_edges = (int(day5_com[1]-50), int(day5_com[1]+50))
    y_edges = ( int(day5_com[0]-50), int(day5_com[0]+50))
    g_img = sess.s2p_ops[len(sess.s2p_stats)-1]['meanImg'][x_edges[0]:x_edges[1], y_edges[0]:y_edges[1]]
    r_img = sess.s2p_ops[len(sess.s2p_stats)-1]['meanImg_chan2'][x_edges[0]:x_edges[1], y_edges[0]:y_edges[1]]
    day5img[:,:,1] = g_img/np.amax(g_img)
    day5img[:,:,0] = r_img/np.amax(r_img)
    day5img[:,:,2] = r_img/np.amax(r_img)
    
    day5_ax = fig.add_subplot(gs[1:5,4])
    day5_ax.imshow(day5img,cmap='Greys_r')
    
    
    circle = plt.Circle((50,50), 7, fill=False, color='blue',linewidth=3)
    day5_ax.add_patch(circle)
    
    fig.suptitle("cell %d" % cell)
    
    if save:
        mouse_dir = os.path.join(figdir, sess.mouse)
        os.makedirs(mouse_dir,exist_ok = True)
        fig.savefig(os.path.join(mouse_dir,"cell%d.pdf" % cell),format = 'pdf')
    

# #     ax[2].scatter(*day0_com)
# #    

/tmp/ipykernel_132003/2890224693.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize = [20, 5])


In [37]:
sess = u.load_single_day(ctrl_mice[3],0)
pc_mask = sess.fam_place_cell_mask() + sess.nov_place_cell_mask() >0
pc_inds = np.argwhere(pc_mask)[:,0]

{'date': '29_11_2020', 'scene': 'YMaze_LNovel', 'session': 2, 'scan': 18, 'novel_arm': 1, 'ravel_ind': 1}



KeyboardInterrupt



In [ ]:
save = False
for cell in pc_inds:
    fig, ax = plt.subplots(1,2, figsize = [10, 3])
    
    fam_trialmat, nov_trialmat = sess.trial_matrices['spks'][sess.trial_info['LR']==-1*sess.novel_arm,:,:][:,:,cell], sess.trial_matrices['spks'][sess.trial_info['LR']==sess.novel_arm,:,:][:,:,cell]
    mean = np.nanmean(sess.trial_matrices['spks'][:,:,cell].ravel())
    fam_trialmat[np.isnan(fam_trialmat)]=1E-3
    nov_trialmat[np.isnan(nov_trialmat)]=1E-3
    h = ax[0].imshow(fam_trialmat/mean, cmap = 'magma', aspect='auto')#, vmin = 0, vmax=10)
    plt.colorbar(h,ax=ax[0])
    h = ax[1].imshow(nov_trialmat/mean, cmap = 'magma', aspect='auto')#, vmin = 0, vmax=10)
    plt.colorbar(h,ax=ax[1])
    ax[0].set_title(f"{cell}")
    
    
    if save:
        mouse_dir = os.path.join(figdir, sess.mouse)
        os.makedirs(mouse_dir,exist_ok = True)
        fig.savefig(os.path.join(mouse_dir,"cell%d.pdf" % cell),format = 'pdf')
    

#     ax[2].scatter(*day0_com)